In [1]:
# http://nbviewer.jupyter.org/github/ricklupton/beamfe/blob/master/theory/FE%20element%20matrices.ipynb
import numpy as np
import sympy
from sympy import symbols, Matrix
sympy.init_printing()
%matplotlib inline

In [2]:
xi, l, rho = symbols('xi, l, rho')

# Shape functions
S = Matrix(np.zeros((4, 12)))
x2 = (1 - xi)
S[0, 0 ] =  x2                     # extension
S[0, 6 ] =  xi
S[1, 1 ] =  x2**2 * (3 - 2*x2)     # y-deflection
S[1, 7 ] =  xi**2 * (3 - 2*xi)
S[1, 5 ] = -x2**2 * (x2 - 1) * l  
S[1, 11] =  xi**2 * (xi - 1) * l  
S[2, 2 ] =  x2**2 * (3 - 2*x2)     # z-deflection
S[2, 8 ] =  xi**2 * (3 - 2*xi)
S[2, 4 ] =  x2**2 * (x2 - 1) * l
S[2, 10] = -xi**2 * (xi - 1) * l
S[3, 3 ] =  x2                     # torsion
S[3, 9 ] =  xi

In [3]:
S[:3, :].T

⎡-ξ + 1          0.0                  0.0        ⎤
⎢                                                ⎥
⎢                2                               ⎥
⎢ 0.0    (-ξ + 1) ⋅(2⋅ξ + 1)          0.0        ⎥
⎢                                                ⎥
⎢                                     2          ⎥
⎢ 0.0            0.0          (-ξ + 1) ⋅(2⋅ξ + 1)⎥
⎢                                                ⎥
⎢ 0.0            0.0                  0.0        ⎥
⎢                                                ⎥
⎢                                            2   ⎥
⎢ 0.0            0.0            -l⋅ξ⋅(-ξ + 1)    ⎥
⎢                                                ⎥
⎢                       2                        ⎥
⎢ 0.0       l⋅ξ⋅(-ξ + 1)              0.0        ⎥
⎢                                                ⎥
⎢  ξ             0.0                  0.0        ⎥
⎢                                                ⎥
⎢            2                                   ⎥
⎢ 0.0       ξ ⋅(-2⋅ξ + 3)      

In [4]:
rho1, rho2 = symbols('rho_1, rho_2')
rho = (1 - xi)*rho1 + xi*rho2
rho

In [5]:
def sym_me():
    m = Matrix(np.diag([rho, rho, rho, 0]))
    integrand = S.T * m * S
    me = integrand.applyfunc(
        lambda xxx: l * sympy.integrate(xxx, (xi, 0, 1)).expand().factor()
    )
    return me
me = sym_me()
me.shape